In [13]:
import cv2
import numpy as np
from numpy.linalg import inv
import constraints
import detect_diff

import importlib
importlib.reload(constraints)
importlib.reload(detect_diff)

from constraints import SCORES
from detect_diff import getDifference

import sys 
print(sys.path)

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15


['/private/var/folders/m5/zysgnrhs03d3hlm12zmmtvh80000gn/T/4af174a2-7d4c-4ccf-b9f0-88c18202f1d1', '/Users/tomonagasutashu/.vscode/extensions/ms-python.python-2020.1.57204/pythonFiles', '/Users/tomonagasutashu/.vscode/extensions/ms-python.python-2020.1.57204/pythonFiles/lib/python', '/Users/tomonagasutashu/anaconda3/lib/python37.zip', '/Users/tomonagasutashu/anaconda3/lib/python3.7', '/Users/tomonagasutashu/anaconda3/lib/python3.7/lib-dynload', '', '/Users/tomonagasutashu/anaconda3/lib/python3.7/site-packages', '/Users/tomonagasutashu/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/tomonagasutashu/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/tomonagasutashu/.ipython']


In [14]:
def alignImages(im1, im2):
 
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
   
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
   
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
   
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
   
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
   
  return im1Reg, h
 
if __name__ == '__main__':

    # Read reference image
    refFilename = "resources/shomen_cropped_inside_eraced.jpg"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

    # Read image to be aligned
    imFilename = "resources/0111.2.jpg"
    print("Reading image to align : ", imFilename);  
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)

    print("Aligning images ...")
    # Registered image will be resotred in imReg. 
    # The estimated homography will be stored in h. 
    imReg, h = alignImages(im, imReference)

    h_inv = np.linalg.inv(h)

    # Write aligned image to disk. 
    outFilename = "outputs/aligned0111.1.jpg"
    print("Saving aligned image : ", outFilename); 
    cv2.imwrite(outFilename, imReg)

    # Print estimated homography
    print("Estimated homography : \n",  h)


Reading reference image :  resources/shomen_cropped_inside_eraced.jpg
Reading image to align :  resources/0111.2.jpg
Aligning images ...
Saving aligned image :  outputs/aligned0111.1.jpg
Estimated homography : 
 [[ 7.32286451e-01 -6.83339206e-02 -1.43938608e+02]
 [-1.23298981e-01  8.38732064e-01 -2.05728946e+01]
 [-5.02519994e-04  2.24377073e-04  1.00000000e+00]]


In [20]:
  # Use inverse homography
  height, width, channels = im2.shape
  h_inv = np.linalg.inv(h)
  img_back = cv2.warpPerspective(im1, h_inv, (width, height))

NameError: name 'im2' is not defined

In [19]:
    # Read background image
    bcgFilename = "resources/0111.0.jpg"
    print("Reading reference image : ", bcgFilename)
    img_bcg = cv2.imread(bcgFilename, cv2.IMREAD_COLOR)

    # Read new image
    newFilename = "resources/0111.2.jpg"
    print("Reading reference image : ", newFilename)
    img_new = cv2.imread(newFilename, cv2.IMREAD_COLOR)

    gray_bcg = cv2.cvtColor(img_bcg, cv2.COLOR_BGR2GRAY)
    gray_new = cv2.cvtColor(img_new, cv2.COLOR_BGR2GRAY)

    diff_thred, gray1, gray2, diff = getDifference(gray_bcg, gray_new)

    print(img_new.shape, diff_thred.shape)
    # apply mask
    dst = np.empty_like(img_new)
    dst[:,:,0] = cv2.bitwise_and(img_new[:,:,0], diff_thred)
    dst[:,:,1] = cv2.bitwise_and(img_new[:,:,1], diff_thred)
    dst[:,:,2] = cv2.bitwise_and(img_new[:,:,2], diff_thred)

    # Write applied mask to disk.
    outFilename = "outputs/applymask.jpg"
    print("Saving detected difference image : ", outFilename)
    cv2.imwrite(outFilename, dst)

    # Write detected difference to disk.
    outFilename = "outputs/diff.jpg"
    print("Saving detected difference image : ", outFilename)
    cv2.imwrite(outFilename, diff_thred)


Reading reference image :  resources/0111.0.jpg
Reading reference image :  resources/0111.2.jpg
(720, 1080, 3) (720, 1080)
Saving detected difference image :  outputs/applymask.jpg
Saving detected difference image :  outputs/diff.jpg


True